# 데이터 준비

In [ ]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 데이터 가져오기

현재 경로에 알집 필요  
unzip 후에 가공 가능하게 데이터 처리


In [ ]:
!unzip -q tatoo1.zip

In [ ]:
import splitfolders

import os

import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf

import pickle

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
folder_path = './tatoo'
label_names = os.listdir(folder_path)
label_names

['old_new_school', 'polynesian', 'ilezumi', 'blackwork']

In [ ]:
dataset = {}

for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'old_new_school': ['/content/tatoo/old_new_school/올스드쿨,뉴스쿨_684.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_137.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_280.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_1052.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_441.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_984.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_858.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_714.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_599.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_493.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_357.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_1107.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_118.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_809.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_772.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_277.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_417.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_692.jpg',
  '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_21

## 디렉토리 생성

이미지가 전처리 이후에 저장될 디렉토리 생성

In [ ]:
!mkdir resized

In [ ]:
!mkdir resized/blackwork
!mkdir resized/ilezumi
!mkdir resized/old_new_school
!mkdir resized/polynesian

In [ ]:
dataset.items()

dict_items([('old_new_school', ['/content/tatoo/old_new_school/올스드쿨,뉴스쿨_684.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_137.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_280.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_1052.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_441.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_984.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_858.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_714.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_599.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_493.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_357.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_1107.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_118.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_809.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_772.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_277.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_417.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_692.jpg', '/content/tatoo/old_new_school/올스드쿨,뉴스쿨_216.jpg', '/content/tatoo/

## 데이터 전처리

### 이미지 리사이즈

In [ ]:
for label, filenames in dataset.items():
    for filename in filenames:
        img = cv2.imread(filename)

        # 이미지의 x, y가 224이 넘을 경우 작게해주기
        percent = 1
        if(img.shape[1] > img.shape[0]) :       # 이미지의 가로가 세로보다 크면 가로를 640으로 맞추고 세로를 비율에 맞춰서
            percent = 224/img.shape[1]
        else :
            percent = 224/img.shape[0]

        img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
                # 이미지 범위 지정
        y,x,h,w = (0,0,img.shape[0], img.shape[1])

        # 그림 주변에 검은색으로 칠하기
        w_x = (224-(w-x))/2  # w_x = (224 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
        h_y = (224-(h-y))/2

        if(w_x < 0):         # 크기가 -면 0으로 지정.
            w_x = 0
        elif(h_y < 0):
            h_y = 0

        M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
        img_re = cv2.warpAffine(img, M, (224, 224)) #이동변환  
       
        # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
        cv2.imwrite('./resized/{0}/{1}'.format(label, filename.split("/")[-1]) , img_re)

In [ ]:
splitfolders.ratio('resized', output='dataset', seed=77, ratio=(0.6, 0.2, 0.2))

Copying files: 4677 files [00:00, 6226.69 files/s]


In [ ]:
folder_path = './dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'old_new_school': ['/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_684.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_137.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_280.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_1052.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_441.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_984.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_714.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_599.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_1107.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_809.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_277.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_417.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_692.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_255.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_290.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_829.jpg',
  '/content/dataset/

In [ ]:
label2index = {'blackwork' : 0, 'ilezumi' : 1 , 'old_new_school' : 2 , 'polynesian' : 3
               }

In [ ]:
x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_train.append(image)
        y_train.append(label2index[label])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = x_train.astype('float32')

In [ ]:
x_train.shape, y_train.shape

((2805, 224, 224, 3), (2805,))

### 데이터 증강

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
folder_path

'/content/dataset/train'

In [ ]:
for label in  os.listdir(folder_path):
    label_path = folder_path + '/' + label + '/'
    for filename in os.listdir(label_path): 
        filepath = label_path + filename

        img = load_img(filepath)
        # img 출력
        # plt.imshow(img)
        # break
        x = img_to_array(img)
        # x.shape 출력
        # print(x.shape)
        # break
        x = x.reshape((1,) + x.shape)

        i = 0
        # flow : augmentation 함수. 계속 반복하므로, break 필요
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=label_path, save_prefix=label, save_format='jpg'):
            i += 1
            if i > 1:
                break  

## train, val, test 데이터 분류

In [ ]:
folder_path = './dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'old_new_school': ['/content/dataset/train/old_new_school/old_new_school_0_8737.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_684.jpg',
  '/content/dataset/train/old_new_school/old_new_school_0_7418.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_137.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_280.jpg',
  '/content/dataset/train/old_new_school/old_new_school_0_5338.jpg',
  '/content/dataset/train/old_new_school/old_new_school_0_122.jpg',
  '/content/dataset/train/old_new_school/old_new_school_0_7490.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_1052.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_441.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_984.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_714.jpg',
  '/content/dataset/train/old_new_school/old_new_school_0_6063.jpg',
  '/content/dataset/train/old_new_school/올스드쿨,뉴스쿨_599.jpg',
  '/content/dataset/train/old_new_school/old_new_school_0_1753.jpg',
  '/content/dataset

In [ ]:
label2index = {'blackwork' : 0, 'ilezumi' : 1 , 'old_new_school' : 2 , 'polynesian' : 3
               }

In [ ]:
x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_train.append(image)
        y_train.append(label2index[label])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train) 

In [ ]:
x_train = x_train.astype('float32')

In [ ]:
x_train.shape, y_train.shape

((8075, 224, 224, 3), (8075,))

validation 데이터와 test 데이터

In [ ]:
folder_path = './dataset/val'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'old_new_school': ['/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_493.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_357.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_216.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_461.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_532.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_444.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_254.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_1000.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_625.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_507.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_629.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_093.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_990.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_618.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_506.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_698.jpg',
  '/content/dataset/val/old_new_school/올스드쿨,뉴스쿨_1138.

In [ ]:
x_val, y_val = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)
        x_val.append(image)
        y_val.append(label2index[label])

In [ ]:
x_val, y_val= np.array(x_val), np.array(y_val)

In [ ]:
x_val = x_val.astype('float32')

In [ ]:
x_val.shape, y_val.shape

((934, 224, 224, 3), (934,))

In [ ]:
folder_path = './dataset/test'
dataset = {}

for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'old_new_school': ['/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_858.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_118.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_772.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_080.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_149.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_894.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_1097.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_1062.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_735.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_1100.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_477.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_761.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_1163.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_627.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_500.jpg',
  '/content/dataset/test/old_new_school/올스드쿨,뉴스쿨_985.jpg',
  '/content/dataset/test/old_new_s

In [ ]:
x_test, y_test = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_test.append(image)
        y_test.append(label2index[label])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)

In [ ]:
x_test = x_test.astype('float32')

In [ ]:
x_test.shape, y_test.shape

((938, 224, 224, 3), (938,))

제로 센터링

In [ ]:
def zero_mean(image):
    return np.mean(image, axis=0)

In [ ]:
zero_mean_img = zero_mean(x_train)

In [ ]:
zero_mean_img.shape

(224, 224, 3)

In [ ]:
x_train -= zero_mean_img

In [ ]:
x_val -= zero_mean_img
x_test -= zero_mean_img

# Training Model

CNN

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 10, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

AlexNet

In [ ]:
# modeling(functional API)
input_shape = (227, 227, 3)  # 논문에서 제시된 shape
x = Input(shape = input_shape, name='INPUT')

# CONV
conv1 = tf.keras.layers.Conv2D(filters=96, kernel_size=11, activation='relu', strides=4, name='CONV_1')(x)
pool1 = MaxPooling2D((3,3), strides=2, name='POOL_1')(conv1)  # overlapped pooling
# lrn1 = local_response_normalization(conv1,depth_radius=5, bias=2, alpha=0.0001, beta=0.75) 
lrn1 = BatchNormalization(name='LRN_1')(pool1)

conv2 = tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation='relu', strides=1, padding='same', name='CONV_2')(lrn1)
pool2 = MaxPooling2D((3,3), strides=2, name='POOL_2')(conv2)
# lrn2 = local_response_normalization(conv2,depth_radius=5, bias=2,  alpha=0.0001, beta=0.75)
lrn2 = BatchNormalization(name='LRN_2')(pool2)

conv3 = tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation='relu', strides=1, padding='same', name='CONV_3')(lrn2)
conv4 = tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation='relu', strides=1, padding='same', name='CONV_4')(conv3)
conv5 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', strides=1, padding='same', name='CONV_5')(conv4)
pool3 = MaxPooling2D((3,3), strides=2, name='POOL_3')(conv5)

# FC
f = Flatten()(pool3)
f = Dense(4096, activation='relu', name='FC_1')(f)
f = Dropout(0.5)(f)  # 논문 parameter 0.5 이용
f = Dense(4096, activation='relu', name='FC_2')(f)
f = Dropout(0.5)(f)
out = Dense(1000, activation='softmax', name='OUTPUT')(f)

model = tf.keras.Model(inputs=x, outputs=out)
model.summary()

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

VGG16

In [ ]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(include_top=False, input_shape=(224, 224,3), weights='imagenet')

#bottleneck_features_train = base_model.predict_generator


for layer in base_model.layers[:-10]: 
    layer.trainable = False			

# base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(128, activation='relu')(x) # 128로 한번 낮춰볼것
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
# 컴파일
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.00005),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
# 콜벡 - 팩토
callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9,
                              patience=1, min_lr=0)

model.fit(x_train, y_train, epochs = 50, validation_data=(x_val, y_val), verbose = 1, callbacks=[callback])

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224, 224,3), weights = 'imagenet')

for layer in base_model.layers[:-10]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 10, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

Resnet 152

In [ ]:
from tensorflow.keras.applications import ResNet152
base_model = ResNet152(include_top=False, input_shape = (300,300,3), weights = 'imagenet')

for layer in base_model.layers[:450]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

GoogleNet

In [ ]:
def create_googlenet(weights_path=None):
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = Input(shape=(3, 224, 224))

    input_pad = ZeroPadding2D(padding=(3, 3))(input)
    conv1_7x7_s2 = Conv2D(64, (7,7), strides=(2,2), padding='valid', activation='relu', 
                          name='conv1/7x7_s2', kernel_regularizer=l2(0.0002))(input_pad)
    pool1_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool1/3x3_s2')(pool1_helper)
    pool1_norm1 = LRN(name='pool1/norm1')(pool1_3x3_s2)

    conv2_3x3_reduce = Conv2D(64, (1,1), padding='same', 
                              activation='relu', name='conv2/3x3_reduce', kernel_regularizer=l2(0.0002))(pool1_norm1)
    conv2_3x3 = Conv2D(192, (3,3), padding='same', 
                       activation='relu', name='conv2/3x3', kernel_regularizer=l2(0.0002))(conv2_3x3_reduce)
    conv2_norm2 = LRN(name='conv2/norm2')(conv2_3x3)
    conv2_zero_pad = ZeroPadding2D(padding=(1, 1))(conv2_norm2)
    pool2_helper = PoolHelper()(conv2_zero_pad)
    pool2_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool2/3x3_s2')(pool2_helper)

    # Inception 3a ############################################################
    inception_3a_1x1 = Conv2D(64, (1,1), padding='same', activation='relu', 
                              name='inception_3a/1x1', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)

    inception_3a_3x3_reduce = Conv2D(96, (1,1), padding='same', activation='relu', 
                                     name='inception_3a/3x3_reduce', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
    inception_3a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3_reduce)
    inception_3a_3x3 = Conv2D(128, (3,3), padding='valid', activation='relu', 
                              name='inception_3a/3x3', kernel_regularizer=l2(0.0002))(inception_3a_3x3_pad)

    inception_3a_5x5_reduce = Conv2D(16, (1,1), padding='same', activation='relu',
                                     name='inception_3a/5x5_reduce', kernel_regularizer=l2(0.0002))(pool2_3x3_s2)
    inception_3a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5_reduce)
    inception_3a_5x5 = Conv2D(32, (5,5), padding='valid', activation='relu', 
                              name='inception_3a/5x5', kernel_regularizer=l2(0.0002))(inception_3a_5x5_pad)
    
    inception_3a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_3a/pool')(pool2_3x3_s2)
    inception_3a_pool_proj = Conv2D(32, (1,1), padding='same', activation='relu', 
                                    name='inception_3a/pool_proj', kernel_regularizer=l2(0.0002))(inception_3a_pool)
    # output : concat
    inception_3a_output = Concatenate(axis=1, 
                                      name='inception_3a/output')([inception_3a_1x1,inception_3a_3x3,
                                                                   inception_3a_5x5,inception_3a_pool_proj])

    # Inception 3b ############################################################
    inception_3b_1x1 = Conv2D(128, (1,1), padding='same', activation='relu', 
                              name='inception_3b/1x1', kernel_regularizer=l2(0.0002))(inception_3a_output)

    inception_3b_3x3_reduce = Conv2D(128, (1,1), padding='same', activation='relu',
                                     name='inception_3b/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_3a_output)
    inception_3b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3_reduce)
    inception_3b_3x3 = Conv2D(192, (3,3), padding='valid', activation='relu', 
                              name='inception_3b/3x3', kernel_regularizer=l2(0.0002))(inception_3b_3x3_pad)
    inception_3b_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_3b/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_3a_output)

    inception_3b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5_reduce)
    inception_3b_5x5 = Conv2D(96, (5,5), padding='valid', activation='relu', 
                              name='inception_3b/5x5', kernel_regularizer=l2(0.0002))(inception_3b_5x5_pad)
    
    inception_3b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_3b/pool')(inception_3a_output)
    inception_3b_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_3b/pool_proj', kernel_regularizer=l2(0.0002))(inception_3b_pool)
    # output : concat
    inception_3b_output = Concatenate(axis=1, 
                                      name='inception_3b/output')([inception_3b_1x1,inception_3b_3x3,
                                                                   inception_3b_5x5,inception_3b_pool_proj])

    inception_3b_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_3b_output)
    pool3_helper = PoolHelper()(inception_3b_output_zero_pad)
    pool3_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool3/3x3_s2')(pool3_helper)


    ###########################################################################
    # Inception 4a ############################################################
    inception_4a_1x1 = Conv2D(192, (1,1), padding='same', activation='relu', 
                              name='inception_4a/1x1', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)

    inception_4a_3x3_reduce = Conv2D(96, (1,1), padding='same', activation='relu', 
                                     name='inception_4a/3x3_reduce', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
    inception_4a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4a_3x3_reduce)
    inception_4a_3x3 = Conv2D(208, (3,3), padding='valid', activation='relu', 
                              name='inception_4a/3x3' ,kernel_regularizer=l2(0.0002))(inception_4a_3x3_pad)

    inception_4a_5x5_reduce = Conv2D(16, (1,1), padding='same', activation='relu', 
                                     name='inception_4a/5x5_reduce', kernel_regularizer=l2(0.0002))(pool3_3x3_s2)
    inception_4a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4a_5x5_reduce)
    inception_4a_5x5 = Conv2D(48, (5,5), padding='valid', activation='relu', 
                              name='inception_4a/5x5', kernel_regularizer=l2(0.0002))(inception_4a_5x5_pad)

    inception_4a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_4a/pool')(pool3_3x3_s2)
    inception_4a_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4a/pool_proj', kernel_regularizer=l2(0.0002))(inception_4a_pool)
    
    # output : concat
    inception_4a_output = Concatenate(axis=1, 
                                      name='inception_4a/output')([inception_4a_1x1,inception_4a_3x3,
                                                                   inception_4a_5x5,inception_4a_pool_proj])

    # auxiliary classifier **********************
    loss1_ave_pool = AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss1/ave_pool')(inception_4a_output)
    loss1_conv = Conv2D(128, (1,1), padding='same', activation='relu', name='loss1/conv', kernel_regularizer=l2(0.0002))(loss1_ave_pool)
    loss1_flat = Flatten()(loss1_conv)
    loss1_fc = Dense(1024, activation='relu', name='loss1/fc', kernel_regularizer=l2(0.0002))(loss1_flat)
    loss1_drop_fc = Dropout(rate=0.7)(loss1_fc)
    loss1_classifier = Dense(1000, name='loss1/classifier', kernel_regularizer=l2(0.0002))(loss1_drop_fc)
    loss1_classifier_act = Activation('softmax')(loss1_classifier)

    # Inception 4b ############################################################
    inception_4b_1x1 = Conv2D(160, (1,1), padding='same', activation='relu',
                              name='inception_4b/1x1', kernel_regularizer=l2(0.0002))(inception_4a_output)

    inception_4b_3x3_reduce = Conv2D(112, (1,1), padding='same', activation='relu', 
                                     name='inception_4b/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4a_output)
    inception_4b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4b_3x3_reduce)
    inception_4b_3x3 = Conv2D(224, (3,3), padding='valid', activation='relu', 
                              name='inception_4b/3x3', kernel_regularizer=l2(0.0002))(inception_4b_3x3_pad)

    inception_4b_5x5_reduce = Conv2D(24, (1,1), padding='same', activation='relu', 
                                     name='inception_4b/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4a_output)
    inception_4b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4b_5x5_reduce)
    inception_4b_5x5 = Conv2D(64, (5,5), padding='valid', activation='relu', 
                              name='inception_4b/5x5', kernel_regularizer=l2(0.0002))(inception_4b_5x5_pad)

    inception_4b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_4b/pool')(inception_4a_output)
    inception_4b_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4b/pool_proj', kernel_regularizer=l2(0.0002))(inception_4b_pool)
    
    # output : concat
    inception_4b_output = Concatenate(axis=1, 
                                      name='inception_4b/output')([inception_4b_1x1,inception_4b_3x3,
                                                                   inception_4b_5x5,inception_4b_pool_proj])

    # Inception 4c ############################################################
    inception_4c_1x1 = Conv2D(128, (1,1), padding='same', activation='relu', 
                              name='inception_4c/1x1', kernel_regularizer=l2(0.0002))(inception_4b_output)

    inception_4c_3x3_reduce = Conv2D(128, (1,1), padding='same', activation='relu', 
                                     name='inception_4c/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4b_output)
    inception_4c_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4c_3x3_reduce)
    inception_4c_3x3 = Conv2D(256, (3,3), padding='valid', activation='relu', 
                              name='inception_4c/3x3', kernel_regularizer=l2(0.0002))(inception_4c_3x3_pad)

    inception_4c_5x5_reduce = Conv2D(24, (1,1), padding='same', activation='relu', 
                                     name='inception_4c/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4b_output)
    inception_4c_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4c_5x5_reduce)
    inception_4c_5x5 = Conv2D(64, (5,5), padding='valid', activation='relu', 
                              name='inception_4c/5x5', kernel_regularizer=l2(0.0002))(inception_4c_5x5_pad)

    inception_4c_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_4c/pool')(inception_4b_output)
    inception_4c_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4c/pool_proj', kernel_regularizer=l2(0.0002))(inception_4c_pool)

    # output : concat
    inception_4c_output = Concatenate(axis=1, 
                                      name='inception_4c/output')([inception_4c_1x1,inception_4c_3x3,
                                                                   inception_4c_5x5,inception_4c_pool_proj])

    # Inception 4d ############################################################
    inception_4d_1x1 = Conv2D(112, (1,1), padding='same', activation='relu', 
                              name='inception_4d/1x1', kernel_regularizer=l2(0.0002))(inception_4c_output)

    inception_4d_3x3_reduce = Conv2D(144, (1,1), padding='same', activation='relu',
                                     name='inception_4d/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4c_output)
    inception_4d_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4d_3x3_reduce)
    inception_4d_3x3 = Conv2D(288, (3,3), padding='valid', activation='relu', 
                              name='inception_4d/3x3', kernel_regularizer=l2(0.0002))(inception_4d_3x3_pad)

    inception_4d_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_4d/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4c_output)
    inception_4d_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4d_5x5_reduce)
    inception_4d_5x5 = Conv2D(64, (5,5), padding='valid', activation='relu', 
                              name='inception_4d/5x5', kernel_regularizer=l2(0.0002))(inception_4d_5x5_pad)

    inception_4d_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_4d/pool')(inception_4c_output)
    inception_4d_pool_proj = Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4d/pool_proj', kernel_regularizer=l2(0.0002))(inception_4d_pool)

    # output : concat
    inception_4d_output = Concatenate(axis=1, 
                                      name='inception_4d/output')([inception_4d_1x1,inception_4d_3x3,
                                                                   inception_4d_5x5,inception_4d_pool_proj])

    # auxiliary classifier **********************
    loss2_ave_pool = AveragePooling2D(pool_size=(5,5), strides=(3,3), name='loss2/ave_pool')(inception_4d_output)
    loss2_conv = Conv2D(128, (1,1), padding='same', activation='relu', name='loss2/conv', kernel_regularizer=l2(0.0002))(loss2_ave_pool)
    loss2_flat = Flatten()(loss2_conv)
    loss2_fc = Dense(1024, activation='relu', name='loss2/fc', kernel_regularizer=l2(0.0002))(loss2_flat)
    loss2_drop_fc = Dropout(rate=0.7)(loss2_fc)
    loss2_classifier = Dense(1000, name='loss2/classifier', kernel_regularizer=l2(0.0002))(loss2_drop_fc)
    loss2_classifier_act = Activation('softmax')(loss2_classifier)

    # Inception 4e ############################################################
    inception_4e_1x1 = Conv2D(256, (1,1), padding='same', activation='relu', 
                              name='inception_4e/1x1', kernel_regularizer=l2(0.0002))(inception_4d_output)

    inception_4e_3x3_reduce = Conv2D(160, (1,1), padding='same', activation='relu',
                                     name='inception_4e/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_4d_output)
    inception_4e_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_3x3_reduce)
    inception_4e_3x3 = Conv2D(320, (3,3), padding='valid', activation='relu', 
                              name='inception_4e/3x3', kernel_regularizer=l2(0.0002))(inception_4e_3x3_pad)

    inception_4e_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_4e/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_4d_output)
    inception_4e_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_4e_5x5_reduce)
    inception_4e_5x5 = Conv2D(128, (5,5), padding='valid', activation='relu', 
                              name='inception_4e/5x5', kernel_regularizer=l2(0.0002))(inception_4e_5x5_pad)

    inception_4e_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_4e/pool')(inception_4d_output)
    inception_4e_pool_proj = Conv2D(128, (1,1), padding='same', activation='relu', 
                                    name='inception_4e/pool_proj', kernel_regularizer=l2(0.0002))(inception_4e_pool)

    # output : concat
    inception_4e_output = Concatenate(axis=1, 
                                      name='inception_4e/output')([inception_4e_1x1,inception_4e_3x3,
                                                                   inception_4e_5x5,inception_4e_pool_proj])

    inception_4e_output_zero_pad = ZeroPadding2D(padding=(1, 1))(inception_4e_output)
    pool4_helper = PoolHelper()(inception_4e_output_zero_pad)
    pool4_3x3_s2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid', name='pool4/3x3_s2')(pool4_helper)

    ###########################################################################
    # Inception 5a ############################################################
    inception_5a_1x1 = Conv2D(256, (1,1), padding='same', activation='relu', 
                              name='inception_5a/1x1', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)

    inception_5a_3x3_reduce = Conv2D(160, (1,1), padding='same', activation='relu', 
                                     name='inception_5a/3x3_reduce', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
    inception_5a_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_5a_3x3_reduce)
    inception_5a_3x3 = Conv2D(320, (3,3), padding='valid', activation='relu', 
                              name='inception_5a/3x3', kernel_regularizer=l2(0.0002))(inception_5a_3x3_pad)

    inception_5a_5x5_reduce = Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_5a/5x5_reduce', kernel_regularizer=l2(0.0002))(pool4_3x3_s2)
    inception_5a_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_5a_5x5_reduce)
    inception_5a_5x5 = Conv2D(128, (5,5), padding='valid', activation='relu', 
                              name='inception_5a/5x5', kernel_regularizer=l2(0.0002))(inception_5a_5x5_pad)

    inception_5a_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_5a/pool')(pool4_3x3_s2)
    inception_5a_pool_proj = Conv2D(128, (1,1), padding='same', activation='relu', 
                                    name='inception_5a/pool_proj', kernel_regularizer=l2(0.0002))(inception_5a_pool)

    # output : concat
    inception_5a_output = Concatenate(axis=1,
                                      name='inception_5a/output')([inception_5a_1x1,inception_5a_3x3,
                                                                   inception_5a_5x5,inception_5a_pool_proj])

    # Inception 5b ############################################################
    inception_5b_1x1 = Conv2D(384, (1,1), padding='same', activation='relu', 
                              name='inception_5b/1x1', kernel_regularizer=l2(0.0002))(inception_5a_output)

    inception_5b_3x3_reduce = Conv2D(192, (1,1), padding='same', activation='relu', 
                                     name='inception_5b/3x3_reduce', kernel_regularizer=l2(0.0002))(inception_5a_output)
    inception_5b_3x3_pad = ZeroPadding2D(padding=(1, 1))(inception_5b_3x3_reduce)
    inception_5b_3x3 = Conv2D(384, (3,3), padding='valid', activation='relu', 
                              name='inception_5b/3x3', kernel_regularizer=l2(0.0002))(inception_5b_3x3_pad)

    inception_5b_5x5_reduce = Conv2D(48, (1,1), padding='same', activation='relu', 
                                     name='inception_5b/5x5_reduce', kernel_regularizer=l2(0.0002))(inception_5a_output)
    inception_5b_5x5_pad = ZeroPadding2D(padding=(2, 2))(inception_5b_5x5_reduce)
    inception_5b_5x5 = Conv2D(128, (5,5), padding='valid', activation='relu', 
                              name='inception_5b/5x5', kernel_regularizer=l2(0.0002))(inception_5b_5x5_pad)

    inception_5b_pool = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_5b/pool')(inception_5a_output)
    inception_5b_pool_proj = Conv2D(128, (1,1), padding='same', activation='relu', 
                                    name='inception_5b/pool_proj', kernel_regularizer=l2(0.0002))(inception_5b_pool)

    # output : concat
    inception_5b_output = Concatenate(axis=1, 
                                      name='inception_5b/output')([inception_5b_1x1,inception_5b_3x3,
                                                                   inception_5b_5x5,inception_5b_pool_proj])

    pool5_7x7_s1 = AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b_output)
    loss3_flat = Flatten()(pool5_7x7_s1)
    pool5_drop_7x7_s1 = Dropout(rate=0.4)(loss3_flat)
    loss3_classifier = Dense(1000, name='loss3/classifier', kernel_regularizer=l2(0.0002))(pool5_drop_7x7_s1)
    loss3_classifier_act = Activation('softmax', name='prob')(loss3_classifier)


   # Final Model : input + output[aux.cls1, aux.cls2, main.cls]
    googlenet = Model(inputs=input, outputs=[loss1_classifier_act,loss2_classifier_act,loss3_classifier_act])

    if weights_path:
        googlenet.load_weights(weights_path)

    if keras.backend.backend() == 'tensorflow':
        # convert the convolutional kernels for tensorflow
        ops = []
        for layer in googlenet.layers:
            if layer.__class__.__name__ == 'Conv2D':
                original_w = K.get_value(layer.kernel)
                converted_w = convert_kernel(original_w)
                ops.append(tf.assign(layer.kernel, converted_w).op)
        K.get_session().run(ops)

    return googlenet